In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import matplotlib.pyplot as plt 
import matplotlib.image as img
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from torch.utils.data import DataLoader, Dataset
from torch.nn import functional as F
from sklearn.metrics import accuracy_score
from torch.autograd import Variable
import torch.utils.data as data
import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder
import torch.optim as optim

In [ ]:
import zipfile

Dataset = "dogs-vs-cats"

# Will unzip the files so that you can see them..
with zipfile.ZipFile("/kaggle/input/"+Dataset+"/test1.zip","r") as z:
    z.extractall(".")

In [ ]:
with zipfile.ZipFile("/kaggle/input/"+Dataset+"/train.zip","r") as z:
    z.extractall(".")

In [ ]:
os.mkdir('train_new')

In [ ]:
os.mkdir('valid_new')

In [ ]:
os.mkdir('./valid_new/dogs')
os.mkdir('./valid_new/cats')

In [ ]:
os.listdir('./train/')

In [ ]:
os.mkdir('./train_new/cats')
os.mkdir('./train_new/dogs')

In [ ]:
import re

In [ ]:
import shutil

In [ ]:
data=os.listdir('./train')
# categories=[]
for i in data:
    category=i.split('.')[0]
#     print(category)
    if category=='dog':
        shutil.copy('./train/'+i,'./train_new/dogs/')
    else:
        shutil.move('./train/'+i,'./train_new/cats/')
# df=pd.DataFrame({
#     'filename':train,
#     'caregory':categories
# })

In [ ]:
data=os.listdir('./train_new/cats/')
# categories=[]
for i in data:
    validationDogsSearchObj = re.search("5\d\d\d", i)
    if validationDogsSearchObj:
        shutil.move('./train_new/cats/'+i, 'valid_new/cats/')

In [ ]:
os.listdir('./train_new/cats/')[1555]

In [ ]:
im = img.imread('./train_new/cats/cat.12167.jpg') 
  
# show image 
plt.imshow(im) 

In [ ]:
input_size=224
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomRotation([-30, 30]),
        transforms.Resize((input_size,input_size)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [ ]:
train_data = torchvision.datasets.ImageFolder(root="./train_new/", transform=data_transforms['train'])

In [ ]:
valid_data = torchvision.datasets.ImageFolder(root="./valid_new/", transform=data_transforms['valid'])

In [ ]:
print(len(train_data),len(valid_data))

In [ ]:
train_dl=DataLoader(train_data, batch_size=64, shuffle=True)
valid_dl=DataLoader(valid_data, batch_size=64, shuffle=True)

In [ ]:
# train_dl[0]
# data_loaders={'train':train_dl,'valid':valid_dl}

In [ ]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.conv1=nn.Conv2d(in_channels=3,out_channels=16,kernel_size=5,stride=1,padding=0)
        self.relu1=nn.ReLU()
        self.maxpool1=nn.MaxPool2d(kernel_size=2)
        self.conv2=nn.Conv2d(in_channels=16,out_channels=32,kernel_size=5,stride=1,padding=0)
        self.relu2=nn.ReLU()
        self.maxpool2=nn.MaxPool2d(kernel_size=2)
        self.conv3=nn.Conv2d(in_channels=32,out_channels=64,kernel_size=5,stride=1,padding=0)
        self.relu3=nn.ReLU()
        self.maxpool3=nn.MaxPool2d(kernel_size=2)
        self.fc=nn.Linear(64*24*24,2)
    def forward(self,x):
        out=self.conv1(x)
        out=self.relu1(out)
        out=self.maxpool1(out)
        
        out = self.conv2(out)
        out = self.relu2(out)
        out=self.maxpool2(out)
        
        out = self.conv3(out)
        out = self.relu3(out)
        out = self.maxpool3(out)
        
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        
        return out

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
net = Network()
net.to(device)
optimizer = optim.SGD(net.parameters(),0.001, momentum=0.7)
criterion  = nn.CrossEntropyLoss()

In [ ]:
net

In [ ]:
for a,b in enumerate(train_dl,0):
    inputs, labels = data
    print(inputs[0].shape)
    break

In [ ]:
net(inputs)

In [ ]:
for epoch in range(25):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_dl, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 50 == 0:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

In [ ]:
net.

In [ ]:
c=torch.zeros([1, 32, 53, 53], dtype=torch.float32)
c.shape

In [ ]:
maxpool1 = nn.MaxPool2d(kernel_size=2)
cnn1 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, stride=1)

In [ ]:
o=cnn1(c)

In [ ]:
m=maxpool1(o)

In [ ]:
o.shape

In [ ]:
m.shape